Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [279]:
import random
from random import choices
from copy import copy
import lab9_lib

# Local Search:
Local search is a heuristic method for solving computationally hard optimization problems. Local search can be used on problems that can be formulated as finding a solution maximizing a criterion among a number of candidate solutions. Local search algorithms move from solution to solution in the space of candidate solutions (the search space) by applying local changes, until a solution deemed optimal is found or a time bound is elapsed.

## Evolutionary Algorithm implementation :


Voglio trovare la combinazione di 0/1 che massimizza la fitness minimizzando il numero di chiamate alla fitness.
Il prof prende combinazioni a caso di 0 e 1 in una stringa da 50 bit e la valuta chiamando la fitness, che, ogni volta che é chiamata, incrementa un contatore.

Idea: con la fitness posso vedere quali pezzi di stringa sono importanti e quali no, e quindi posso preservare i pezzi importanti e buttare via quelli inutili.

Step One: 
  - Generate the initial population of individuals randomly. (First generation)

Step Two: 
  - Repeat the following regenerational steps until termination:
    - Evaluate the fitness of each individual in the population (time limit, sufficient fitness achieved, etc.)
    - Select the fittest individuals for reproduction. (Parents)
    - Breed new individuals through crossover and mutation operations to give birth to offspring.
    - Replace the least-fit individuals of the population with new individuals.

In [280]:
l = 1000
problems = [1, 2, 5, 10]
half_pop_size = 5
µ = 2 * half_pop_size

In [281]:
def init_population():
    return [(choices([0, 1], k=l), 0.0) for _ in range(µ)]

def evaluate_population(population, fitness):
    return [(individual[0], fitness(individual[0])) for individual in population]

def select_with_replacement(population):
    # select a random individual from the population
    return random.choice(population)

def crossover(parent1, parent2):
    # a two point crossover for now
    v = parent1[0]
    w = parent2[0]
    c = random.randint(0, l)
    d = random.randint(0, l)
    if c > d:
        c, d = d, c
    if c != d:
        v[c:d], w[c:d] = w[c:d], v[c:d]
    return (v, 0.0), (w, 0.0)

def mutate(individual):
    # bit flip mutation for now
    p = 0.5
    v = individual[0]
    for i in range(l):
        if p >= random.random():
            v[i] = 1 - v[i]
    return individual


In [282]:
def genetic_algorithm(fitness):
    Best = None
    # 1. Initialize population
    population = init_population()
    population = evaluate_population(population, fitness)
    # 2. Repeat
    for i in range(100):
        for p in population:
            if Best is None or p[1] > Best[1]:
                Best = p
        
        if Best is not None and Best[1]==1:
            break
        
        q = list()
        for _ in range(µ//2):
            # 2.1 Select parents
            parent_a = select_with_replacement(population)
            parent_b = select_with_replacement(population)
            # 2.2 Crossover
            child_a, child_b = crossover(copy(parent_a), copy(parent_b))
            
            # 2.3 Mutate
            mutated_a = mutate(child_a)
            mutated_b = mutate(child_b)
            q.append(mutated_a)
            q.append(mutated_b)
            
        population = evaluate_population(q, fitness)
    
    # 4. Return best individual
    return Best
my_list = list()

for prob in problems:
    fitness = lab9_lib.make_problem(prob)
    b = genetic_algorithm(fitness)
    my_list.append((prob, b[1], fitness.calls))

for m in my_list:
    print(f"Problem\t{m[0]}:\t{m[1]:.2%},\tCalls:\t{m[2]}")

Problem	1:	53.40%,	Calls:	1010
Problem	2:	50.40%,	Calls:	1010
Problem	5:	30.13%,	Calls:	1010
Problem	10:	24.47%,	Calls:	1010


In [283]:
fitness = lab9_lib.make_problem(1)
for n in range(10):
    ind = choices([0, 1], k=1000)
    print(f"{''.join(str(g) for g in ind[0:5])}...: {fitness(ind):.2%}")

print(fitness.calls)

11110...: 51.00%
00001...: 51.50%
00100...: 51.40%
00011...: 46.60%
01110...: 50.10%
00100...: 48.10%
10101...: 49.40%
00010...: 49.60%
00111...: 50.10%
10000...: 50.00%
10


In [284]:
fitness = lab9_lib.make_problem(1)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

10110000110000110111001111010001111010111101000100: 52.00%
11101111011111111011110100010110000011000000001001: 54.00%
01101010111000111001011010100101100101011101110011: 56.00%
11100101011100010101101110110000000111001001100101: 50.00%
11010011100010011101110010110001010000100100010000: 42.00%
10001000001000111110111110110101000001001010010101: 46.00%
10000000110101000001100001110110111101111101000100: 46.00%
11100100000111100111110001110110111100001001101000: 52.00%
00110101010011010001011000111000101110000100011100: 44.00%
00011010110001101010010001100111100110010000011001: 44.00%
10
